# AI Job Dashboard with Plotly Dash
## ABB #2 - Session 1

Code authored by: Shaw Talebi

### imports

In [1]:
import pandas as pd
import re
from collections import Counter

import plotly.express as px
from dash import dcc, html, Dash

### 1) import data

In [2]:
df = pd.read_csv("data/ai_job_data.csv")
df = df.dropna()
df.head()

,company_name,job_title,job_description,salary_min,salary_max
0,murmuration,Staff Software Engineer,Who We Are Murmuration is a nonprofit organiza...,135000.0,165000.0
2,Endeavour Group,Mid/Senior Data Scientist,Company DescriptionLet’s create a more sociabl...,91774.0,170438.0
3,IBM,Data Scientist-Artificial Intelligence,IntroductionIn this role you will join IBM Con...,74567.0,138482.0
4,NielsenIQ,Senior Data Scientist,Company DescriptionGfk is seeking a Data Scien...,57359.0,106524.0
5,Redhorse,Senior Full Stack Software Engineer - AI ML M...,About the OrganizationNow is a great time to j...,141372.0,262548.0


### 2) pre-process data

#### standardize job titles

In [3]:
def standardize_job_title(title):
    """
    Standardizes a given job title string by returning a predefined canonical job title without adding seniority levels.

    Args:
        title (str): The job title string to be standardized.

    Returns:
        str: A standardized job title based on predefined categories, handling compound roles
             and specialized AI-related job functions.
    """
    title = title.lower()
    
    # Define keyword mappings for specialized roles
    role_mappings = {
        "AI Scientist": ["ai scientist", "ml scientist", "ai/ml scientist"],
        "Prompt Engineer": ["prompt engineer"],
        "Deep Learning Engineer": ["deep learning", "dl engineer", "deep learning systems"],
        "Data Scientist": ["data scientist", "decision scientist", "analytics scientist"],
        "Decision Science Analyst": ["decision science", "decision analyst"],
        "Software Engineer": ["software engineer", "full stack", "backend", "frontend", "performance engineer"],
        "Data Engineer": ["data engineer", "etl developer", "pipeline engineer", "spark", "scala", "aws"],
        "ML Ops Engineer": ["ml ops", "model development", "ml operations", "applied data science"],
        "Security Engineer": ["security engineer", "data security"],
        "Electrical Engineer": ["electrical design", "data center design"],
        "Manager": ["manager", "product manager", "project manager"],
        "Director": ["director", "head of", "vp", "vice president"],
        "Intern": ["intern", "trainee"],
        "Researcher": ["researcher", "research engineer"],
    }

    # Check for keywords in the title and return the corresponding standardized title
    for standard_title, keywords in role_mappings.items():
        if any(keyword in title for keyword in keywords):
            return standard_title
    
    # Return "Other" if no match is found
    return "Other"

In [4]:
# Apply the regex-based function to the job_title column
df['standardized_job_title'] = df['job_title'].apply(standardize_job_title)

print("Num raw job titles:", df['job_title'].nunique())
print("Num standardized job titles:", df['standardized_job_title'].nunique())

Num raw job titles: 45
Num standardized job titles: 11


In [5]:
# sum max salary for each standardized job title
s_jobs = df.groupby('standardized_job_title')['salary_max'].mean()
s_jobs = s_jobs.sort_values()

# convert to dataframe
df_jobs = s_jobs.reset_index()
df_jobs.columns = ["Job Title", "Mean Salary"]
df_jobs.head()

,Job Title,Mean Salary
0,Other,70230.666667
1,Prompt Engineer,112071.666667
2,Manager,116800.000000
3,Data Engineer,136379.222222
4,AI Scientist,147840.000000


#### extract common skills from JD

In [6]:
def extract_skills(description):
    """
    Extracts AI-related skills from a given job description.

    Args:
        description (str): The job description text to search for skills.

    Returns:
        list: A list of skills found in the job description, matched from a predefined set of common AI-related skills.

    Notes:
        - The function defines a list of common AI-related skills, including programming languages, frameworks,
          cloud platforms, and statistical concepts.
        - The input description is converted to lowercase to ensure case-insensitive matching.
        - Skills are detected using regular expressions to match whole words, avoiding partial matches (e.g., 
          "spark" will not match "sparking").
    """
    
    # Define a list of common AI-related skills
    skills_list = [
        "python", "r", "java", "c++", "sql", "scala", "spark", "hadoop", "tensorflow", "pytorch",
        "keras", "scikit-learn", "machine learning", "deep learning", "nlp", "natural language processing",
        "computer vision", "data analysis", "data engineering", "big data", "ai", "artificial intelligence",
        "cloud", "aws", "azure", "gcp", "docker", "kubernetes", "linux", "flask", "django", "pandas",
        "numpy", "matplotlib", "seaborn", "plotly", "etl", "api", "statistics", "probability", "regression",
        "classification", "clustering", "time series", "neural networks", "bayesian methods", "git", "mlops"
    ]

    description = description.lower()
    found_skills = [skill for skill in skills_list if re.search(rf"\b{re.escape(skill)}\b", description)]
    
    return found_skills

# notice we could have done something similar to get standardized job titles

In [7]:
# Apply the function to extract skills from each job description
df['extracted_skills'] = df['job_description'].apply(lambda x: extract_skills(str(x)))
df['extracted_skills'].head()

0                  [aws, docker, kubernetes, api, git]
2    [python, sql, machine learning, data engineeri...
3    [python, r, sql, scala, tensorflow, pytorch, m...
4    [python, r, sql, spark, big data, cloud, azure...
5    [python, java, ai, artificial intelligence, ku...
Name: extracted_skills, dtype: object

In [8]:
# create a list with all the skills from the JDs
all_skills = [skill for skills in df['extracted_skills'] for skill in skills]

# count skill occurances
skill_counts = Counter(all_skills)

# Convert the skill counts to a DataFrame
df_skills = pd.DataFrame(skill_counts.items(), columns=["Skill", "Count"]).sort_values(by="Count")
df_skills.head()

,Skill,Count
38,clustering,1
26,mlops,1
37,neural networks,1
27,keras,1
36,c++,1


### 3) create dashboard

In [9]:
# Create the plots
bar_chart = dcc.Graph(
    id='top-roles',
    figure=px.bar(df.sort_values(by='salary_max', ascending=False), 
                  x='standardized_job_title', 
                  y='salary_max', 
                  color='company_name', 
                  title='Highest Paying AI Jobs',
                  labels={'salary_max': 'Maximum Salary', 'job_title': 'Job Title'},
                  height=500)
)

top_jobs_chart = dcc.Graph(
    id='top-jobs-chart',
    figure=px.bar(
        df_jobs[-5:],
        y='Job Title',      
        x='Mean Salary',
        title='Top 5 Roles',
        height=250
    ).update_layout(
        xaxis_title_font_size=12,    # Reduce x-axis label font size
        yaxis_title_font_size=12,    # Reduce y-axis label font size
        xaxis_tickfont_size=10,      # Reduce x-axis tick label font size
        yaxis_tickfont_size=10       # Reduce y-axis tick label font size
    )
)

top_skills_chart = dcc.Graph(
    id='top-skills-chart',
    figure=px.bar(
        df_skills[-5:],
        y='Skill',      
        x='Count',
        title='Top 5 Skills',
        height=250
    ).update_layout(
        xaxis_title_font_size=12,    # Reduce x-axis label font size
        yaxis_title_font_size=12,    # Reduce y-axis label font size
        xaxis_tickfont_size=10,      # Reduce x-axis tick label font size
        yaxis_tickfont_size=10       # Reduce y-axis tick label font size
    )
)

In [10]:
# Initialize the Dash app
app = Dash(__name__)

# App layout with styled divs for positioning
app.layout = html.Div([
    html.Div(bar_chart, style={'width': '70%', 'display': 'inline-block', 'vertical-align': 'top'}),
    html.Div([
        html.Div(top_jobs_chart, style={'height': '50%'}),
        html.Div(top_skills_chart, style={'height': '50%'})
    ], style={'width': '30%', 'display': 'inline-block', 'vertical-align': 'top'})
])

# Run the app
app.run(jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/
